In [18]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_daily_stock_price_history(ticker):
    """
    Fetch the daily stock price history for a given stock ticker using Yahoo Finance.

    Args:
        ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).

    Returns:
        pd.DataFrame: A DataFrame containing the stock price history with columns for Open, High, Low, Close, Volume, etc.
                      Returns None if the ticker is invalid or data is unavailable.
    """
    try:
        # Calculate the date range (1 year before today to today)
        end_date = datetime.now().strftime('%Y-%m-%d')
        start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

        # Fetch the stock data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        # Check if data is fetched successfully
        if stock_data.empty:
            print(f"No data found for ticker '{ticker}'. Please check the ticker symbol or the date range.")
            return None

        # Drop the "Date" index name and "Ticker" column if present
        stock_data.index.name = None
        if 'Ticker' in stock_data.columns:
            stock_data = stock_data.drop(columns=['Ticker'])

        stock_data.columns = stock_data.columns.droplevel('Ticker')
        # Return the cleaned stock data
        return stock_data.iloc[:100]

    except Exception as e:
        return f"An error occurred while fetching data for ticker '{ticker}': {e}"

# Example usage
if __name__ == "__main__":
    ticker = "AAPL"  # Replace with the desired stock ticker

    stock_history = get_daily_stock_price_history(ticker)

    if stock_history is not None:
        print(stock_history.head())

Price            Close        High         Low        Open    Volume
2024-01-25  193.223404  195.313172  192.168574  194.268288  54822100
2024-01-26  191.481918  193.810507  191.004263  193.322905  44594000
2024-01-29  190.795273  191.262983  188.655761  191.073907  47145600
2024-01-30  187.123276  190.864955  186.556062  190.009147  55859400
2024-01-31  183.501022  186.187872  183.451278  186.128151  55467800


In [70]:
from crewai import Agent, Task, Crew, Process
from interpreter import interpreter
from langchain.tools import tool
from langchain_openai import ChatOpenAI

# 1. Configuration and Tools
llm = ChatOpenAI(model="gpt-4-turbo-preview")
interpreter.auto_run = True
interpreter.llm.model = "openai/gpt-4-turbo-preview"

class CLITool:
    @tool("Executor")
    def execute_cli_command(command: str):
        """Create and Execute code using Open Interpreter."""
        result = interpreter.chat(command)
        return result

# 2. Creating an Agent for CLI tasks
cli_agent = Agent(
    role='Software Engineer',
    goal='Always use Executor Tool. Ability to perform CLI operations, write programs and execute using Exector Tool',
    backstory='Expert in command line operations, creating and executing code.',
    tools=[CLITool.execute_cli_command],
    verbose=True,
    llm=llm 
)

# 3. Defining a Task for CLI operations
cli_task = Task(
    description='Identify the OS and then empty my recycle bin',
    agent=cli_agent,
    tools=[CLITool.execute_cli_command]
)

# 4. Creating a Crew with CLI focus
cli_crew = Crew(
    agents=[cli_agent],
    tasks=[cli_task],
    process=Process.sequential,
    manager_llm=llm
)

# 5. Run the Crew
result = cli_crew.kickoff()
print(result)

LLM value is an unknown object


ValidationError: 2 validation errors for Task
expected_output
  Field required [type=missing, input_value={'description': 'Identify...t 0x000002BCB113F6A0>)]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=StructuredTool(name='Exec... at 0x000002BCB113F6A0>), input_type=StructuredTool]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type